In [35]:
import numpy as np
import qutip
from qutip import *
from numpy.random import rand, randint
from qutip.measurement import measure, measurement_statistics
# from qutip.states import ket2dm, basis

In [41]:
np.sqrt(16)

def loss(tree):
    return 0

M = 4 # dimension of Hilbert space. This is a truncated Fock state, with the basis (|0>, |1>, ... |M-1>).
N = 2 # depth of circuit
beta = 1e-5 # this is a small parameter to ensure the coherent states are "weak"
# six states to distinguish between. these are also the ones injected, i think
code = {
    0: qutip.states.coherent(M, beta*np.exp(1j * np.atan(0.5))),
    1: qutip.states.coherent(M, beta*np.exp(-1j * np.atan(0.5)))
}

projs = [qutip.states.ket2dm(qutip.states.basis(M, j)) for j in range(M)] # single-mode projectors
projs0 = [tensor(p, identity(M)) for p in projs] # only measures mode 0
# this gives list of N projection operators
measure(code[0], projs0)

print(code)

# takes an input state, and a decision tree. yields the output state
def process(state, tree):
    for j in range(N):
        anc = code[np.randint(6)]
        state = tensor(state, anc)
        alpha = tree(x)
        
        #k = meas


ValueError: op and state dims should be compatible when state is a ket